In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install transformers
import transformers
from transformers import DistilBertTokenizerFast
from transformers import TFDistilBertForSequenceClassification

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 91.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 81.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 24.1 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
import random
import os
import gc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import pandas as pd
import numpy as np
import random
import os
import gc
from PIL import Image
from textwrap import wrap
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib as mpl
import bz2
import re
import tensorflow as tf
from tensorflow.python.keras import models, layers, optimizers
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.convolutional import Conv1D
from keras.layers import Embedding
from keras.layers import GlobalMaxPooling1D
from keras.layers.core import Activation, Dropout, Dense
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from sklearn.model_selection import train_test_split
from transformers import DistilBertConfig, TFDistilBertForSequenceClassification

In [4]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tokenizers import BertWordPieceTokenizer

import transformers
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import InputExample, InputFeatures

import logging
transformers.logging.set_verbosity_error()

In [5]:
df = pd.read_csv('/content/drive/MyDrive/irfinalproject/file.csv')

In [6]:
df

,Unnamed: 0,Label,Text,ProductId,UserId,Username,HelpfulnessNumerator,HelpfulnessDenominator,Score
0,300433,0,Yawn ... more anti-South cliches: Although the...,B000J2DHJA,A14HIHZX4LX3HA,"J. Wiley ""aka Angry CPU""",1.0,2.0,1.0
1,559025,0,Rated vs. Unrated: Don't make the mistake I ma...,B000REI2X6,A27VSP7HYF10DZ,Linda G. Smith,1.0,1.0,2.0
2,333148,0,"Maybe its just me, but: I didn't even get past...",B002CO180S,ABI9COIFMI7R8,"Kimberly A. Kempf ""momofone""",0.0,0.0,2.0
3,79440,0,Ouch! I've read term papers more interesting t...,B003TNANSO,A32FL8K77UAEN8,Ben Stupar,0.0,0.0,3.0
4,28605,0,Draco would not approve of this.: I am not som...,B00269YMNE,A3RWCDM8NY03SD,"Carol Robinson ""read too much""",2.0,2.0,2.0
...,...,...,...,...,...,...,...,...,...
49995,46521,1,This product tastes just like regular peanut b...,B002GJ9JWS,A2U5A10E93A1S0,AJürckse,1.0,2.0,5.0
49996,450656,1,I have completely changed my diet to healthy e...,B0046HGTJ4,A1DG26ODRXIXBC,madame_dragon,0.0,0.0,5.0
49997,561485,0,I really like other coffee's and hot chocolate...,B000O393QK,A3MAFSTILQSOXR,"kiwi500 ""kiwi500""",0.0,0.0,3.0
49998,427752,1,I could spend a lot of time writing a very glo...,B0031VDJZS,A352U99Q6I1FE7,M.T.,0.0,0.0,5.0


### BERT

In [18]:
from sklearn.model_selection import train_test_split
train_labels, test_labels, train_texts , test_texts = train_test_split(df['Label'][:36000],df['Text'][:36000] ,
                                   random_state=104, 
                                   test_size=0.25, 
                                   shuffle=True)

In [19]:
NON_ALPHANUM = re.compile(r'[\W]')
NON_ASCII = re.compile(r'[^a-z0-1\s]')
def normalize_texts(texts):
    normalized_texts = []
    for text in texts:
        lower = text.lower()
        no_punctuation = NON_ALPHANUM.sub(r' ', lower)
        no_non_ascii = NON_ASCII.sub(r'', no_punctuation)
        normalized_texts.append(no_non_ascii)
    return normalized_texts
        
train_texts = normalize_texts(train_texts)
test_texts = normalize_texts(test_texts)

In [20]:
train_texts_Bert = train_texts
train_labels_Bert = train_labels
test_texts_Bert = test_texts
test_labels_Bert = test_labels
train_reviews = train_texts

In [10]:
train_texts_Bert

['cuisinart dlc 00chb food processor  i purchased this food processor about  months ago and i m still loving it  it is easy to use  easy to clean and just the right size  not to mention it looks great on my counter  i would definitely recommend it ',
 'a careless cd  this cd is not bad at all  but this cd make the music of india something terrible  like demons  i really don t like this so called india s music cd ',
 'great value  great optics but the plastic mount makes it feel a bit unsubstantial  the only reason for the  stars   you will  however  be very impressed with the sharpness and the blurr factor when used at fast apertures  it is a great portrait lens  if you have a canon ef mount  you just cannot justify not owning it when it is under  100   unless you have an  l  class lens to cover this range  in which case you probably wouldn t be reading this review ',
 'very elegant  i was surprised as to how sturdy but elegant the material is  i was going without it at first  but glad

In [21]:
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_texts, train_labels, random_state=57643892, test_size=0.2)


In [22]:
MAX_FEATURES = 12000
#Tokenize texts
tokenizer = Tokenizer(num_words=MAX_FEATURES)
tokenizer.fit_on_texts(train_texts)
train_texts = tokenizer.texts_to_sequences(train_texts)
val_texts = tokenizer.texts_to_sequences(val_texts)
test_texts = tokenizer.texts_to_sequences(test_texts)

In [23]:
MAX_LENGTH = max(len(train_ex) for train_ex in train_texts)
#Add padding
train_texts = pad_sequences(train_texts, maxlen=MAX_LENGTH)
val_texts = pad_sequences(val_texts, maxlen=MAX_LENGTH)
test_texts = pad_sequences(test_texts, maxlen=MAX_LENGTH)

In [24]:
train_texts_Bert, val_texts_Bert, train_labels_Bert, val_labels_Bert = train_test_split(
    train_texts_Bert, train_labels_Bert, random_state=57643892, test_size=0.2)

In [25]:
#Assign tokenizer object to the tokenizer class
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [30]:
train_texts_Bert

['studio gansta  i think this album waz hella weak  dis fool s only a studio gansta  i bet if he came  tha town he s get jacked  he can t do it no mo   he s just weak  and also  the only good song on dis album is ghetto vet  and dat s not even dat good  if u wanna get a good album  don t get dis or tha peace one  i heard dat   it was even weaker  a word of advise  don t get dis album ',
 'this cd is really  really bad    really   the only thing more depressing than listening to bert s make believe phoney suicidal ramblings  is the fact that bert still hasn t committed suicide  because then this album would have never been released  saving every asinine 1 year old teenager from forcing themselves to like it so they can tell everyone at school about how killer the new used cd is  how edgy  do yourself a favor  download i caught fire  all i got  and light with a sharpened edge  because they re the only tolerable songs on this album  it really sucks  really ',
 'exactly what features are o

In [31]:
train_encodings = tokenizer(train_texts_Bert,
                            truncation=True,
                            padding=True,
                            max_length=256)
val_encodings = tokenizer(val_texts_Bert,
                          truncation=True,
                          padding=True,
                          max_length=256)
train_dataset = tf.data.Dataset.from_tensor_slices((
                            dict(train_encodings),
                            train_labels_Bert
                            ))
val_dataset = tf.data.Dataset.from_tensor_slices((
                            dict(val_encodings),
                            val_labels_Bert
                            ))
test_encodings = tokenizer(test_texts_Bert,
                          truncation=True,
                          padding=True,
                          max_length=256)
test_dataset = tf.data.Dataset.from_tensor_slices((
                            dict(test_encodings),
                            test_labels_Bert
                            ))

In [32]:
bert_model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=2)

In [33]:
print(bert_model.summary())

Model: "tf_distil_bert_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 distilbert (TFDistilBertMai  multiple                 66362880  
 nLayer)                                                         
                                                                 
 pre_classifier (Dense)      multiple                  590592    
                                                                 
 classifier (Dense)          multiple                  1538      
                                                                 
 dropout_39 (Dropout)        multiple                  0         
                                                                 
Total params: 66,955,010
Trainable params: 66,955,010
Non-trainable params: 0
_________________________________________________________________
None


In [34]:
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5, epsilon=1e-08)
bert_model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=["binary_accuracy"])
bert_model.fit(train_dataset.shuffle(100).batch(16), 
          epochs=2,
          batch_size=16,
          validation_data=val_dataset.shuffle(100).batch(16))

Epoch 1/2
1350/1350 [==============================] - 621s 438ms/step - loss: 0.4379 - binary_accuracy: 0.8333 - val_loss: 0.4562 - val_binary_accuracy: 0.8245
Epoch 2/2
1350/1350 [==============================] - 581s 430ms/step - loss: 0.3534 - binary_accuracy: 0.8926 - val_loss: 0.3460 - val_binary_accuracy: 0.8787


In [35]:
results_bert = bert_model.evaluate(test_dataset.batch(8))

1125/1125 [==============================] - 80s 70ms/step - loss: 0.3581 - binary_accuracy: 0.8762


In [36]:
bert_model.save_weights('/content/drive/MyDrive/irfinalproject/bert.h5')

In [37]:
bert_model1 = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=2)

In [38]:
bert_model1.load_weights('/content/drive/MyDrive/irfinalproject/bert.h5')

In [39]:
query = " suckssss  "
encoding = tokenizer.encode_plus(query, max_length=MAX_LENGTH, truncation=True, padding='max_length', return_tensors='tf')
outputs = bert_model1(encoding)
#rel_val = outputs.logits.numpy()[0]
predicted_label = outputs.logits.numpy()[0] > 0.5  # assuming threshold of 0.5
print(predicted_label[0])

False


In [40]:
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

# Make predictions on test dataset using the trained BERT model
y_pred = bert_model.predict(test_dataset.batch(8)).logits.argmax(axis=1)
y_true = test_labels_Bert

# Calculate accuracy
acc = accuracy_score(y_true, y_pred)
print('Accuracy:', acc)

# Calculate F1 score
f1 = f1_score(y_true, y_pred)
print('F1 score:', f1)

# Calculate ROC AUC
y_prob = bert_model.predict(test_dataset.batch(8)).logits[:, 1]
roc_auc = roc_auc_score(y_true, y_prob)


1125/1125 [==============================] - 81s 71ms/step
Accuracy: 0.35433333333333333
F1 score: 0.004454342984409799
1125/1125 [==============================] - 76s 67ms/step


In [41]:
print("roc_auc: ",roc_auc)

roc_auc:  0.9653045701673793
